In [ ]:
import os
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import layers
from time import strftime
from scipy.signal import spectrogram, stft, istft

In [ ]:
MODEL_NAME = "GanPlayground3comp"
BUFFER_SIZE = 1000
BATCH_SIZE = 128
NUM_EXAMPLES_TO_GENERATE = 1
NOISE_DIM = 100
STEAD_PATH_DB = "/home/jarek/github/saigon/data/STEAD-processed-stft-microseism-multi.hdf5"

In [ ]:
def plot_all(do, label, fs=100, nperseg=155, file_path=None):
    d0 = pd.DataFrame(data=do[0])
    d1 = pd.DataFrame(data=do[1])
    d2 = pd.DataFrame(data=do[2])

    fig = plt.figure(figsize=(16, 10), dpi=80)
    ax1 = plt.subplot2grid((5, 6), (0, 0), colspan=3)
    ax2 = plt.subplot2grid((5, 6), (1, 0), colspan=3)
    ax3 = plt.subplot2grid((5, 6), (2, 0), colspan=3)
    ax4 = plt.subplot2grid((5, 6), (0, 3), colspan=3)
    ax5 = plt.subplot2grid((5, 6), (1, 3), colspan=3)
    ax6 = plt.subplot2grid((5, 6), (2, 3), colspan=3)
    ax7 = plt.subplot2grid((5, 6), (3, 0), colspan=2, rowspan=2)
    ax8 = plt.subplot2grid((5, 6), (3, 2), colspan=2, rowspan=2)
    ax9 = plt.subplot2grid((5, 6), (3, 4), colspan=2, rowspan=2)

    plt.subplots_adjust(hspace=1, wspace=1)

    sns.lineplot(data=d0, ax=ax1, linewidth=1, legend=None)
    sns.lineplot(data=d1, ax=ax2, linewidth=1, legend=None)
    sns.lineplot(data=d2, ax=ax3, linewidth=1, legend=None)

    ax1.set_title("Vertical component waveform")
    ax1.set(xlabel="Samples", ylabel="Amplitude counts")
    ax1.locator_params(nbins=6, axis="y")

    ax2.set_title("North component waveform")
    ax2.set(xlabel="Samples", ylabel="Amplitude counts")
    ax2.locator_params(nbins=6, axis="y")

    ax3.set_title("East component waveform")
    ax3.set(xlabel="Samples", ylabel="Amplitude counts")
    ax3.locator_params(nbins=6, axis="y")

    f_0, t_0, Sxx_0 = spectrogram(x=do[0], fs=fs)
    f_1, t_1, Sxx_1 = spectrogram(x=do[1], fs=fs)
    f_2, t_2, Sxx_2 = spectrogram(x=do[2], fs=fs)

    ax4.clear()
    ax4.set_title("Vertical component spectrogram")
    ax4.pcolormesh(t_0, f_0, Sxx_0, shading="gouraud")
    ax4.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    ax5.clear()
    ax5.set_title("North component spectrogram")
    ax5.pcolormesh(t_1, f_1, Sxx_1, shading="gouraud")
    ax5.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    ax6.clear()
    ax6.set_title("East component spectrogram")
    ax6.pcolormesh(t_2, f_2, Sxx_2, shading="gouraud")
    ax6.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    f_sftt_0, t_sftt_0, Zxx_0 = stft(
        do[0], window="hanning", fs=fs, nperseg=nperseg
    )
    f_sftt_1, t_sftt_1, Zxx_1 = stft(
        do[1], window="hanning", fs=fs, nperseg=nperseg
    )
    f_sftt_2, t_sftt_2, Zxx_2 = stft(
        do[2], window="hanning", fs=fs, nperseg=nperseg
    )

    ax7.clear()
    ax7.set_title("Vertical component STFT")
    ax7.pcolormesh(t_sftt_0, f_sftt_0, np.abs(Zxx_0), shading="auto")

    ax8.clear()
    ax8.set_title("North component STFT")
    ax8.pcolormesh(t_sftt_1, f_sftt_1, np.abs(Zxx_1), shading="auto")

    ax9.clear()
    ax9.set_title("East component STFT")
    ax9.pcolormesh(t_sftt_2, f_sftt_2, np.abs(Zxx_2), shading="auto")

    plt.suptitle(label, fontsize=14)

    if file_path != None:
        plt.savefig(file_path)
        plt.close(fig)

In [ ]:
def plot_stft(stream, fs=100, nperseg=155):
    f, t, Zxx = stft(stream, window='hanning', fs=fs, nperseg=nperseg)
    # plt.specgram(x_1[0][0], cmap='plasma', Fs=100)
    plt.pcolormesh(t, f, np.abs(Zxx), shading='auto')

In [ ]:
def get_stft_data(file_path, arr_length):
    with h5py.File(file_path, "r") as f:
        keys = f["keys"][:arr_length]
        data = f["data"][:arr_length]
        return (keys, data)

# Read processed data

In [ ]:
(keys, x_train) = get_stft_data(
    STEAD_PATH_DB, 50000
)

# Convert streams to STFT

In [ ]:
# STFT of the stream and then reverse STFT back into original stream
# f, t, Zxx = stft(x_1[1][0], window='hanning', fs=100, nperseg=155)
# k2 = istft(Zxx, window='hanning', fs=100, nperseg=155)

# Scale and reshape data

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 78, 78, 3)

In [ ]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices(x_train)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [ ]:
train_dataset

# Logs and Tensorboard

In [ ]:
folder_name = f"{MODEL_NAME} at {strftime('%H:%M')}"
log_dir = os.path.join("../log/", folder_name)

try:
    os.makedirs(log_dir)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created dirs!")

# Define GAN

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(3 * 3 * 128, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((3, 3, 128)))

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(1, 1), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(2, 2), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(13, 13), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            1,
            (10, 10),
            strides=(1, 1),
            padding="same",
            use_bias=False,
            activation="tanh",
        )
    )
    return model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([BATCH_SIZE, NOISE_DIM], stddev=10e5)
generated_stft = generator(noise, training=False)

generated_stft.shape

In [ ]:
generator.summary()

In [ ]:
tf.keras.utils.plot_model(generator, show_shapes=True, to_file="out/generator.png")

In [ ]:
inversed = istft(generated_stft[0, :, :, 0], window='hanning', fs=100, nperseg=155)
plot_all(inversed[1][:6000], "GAN Generator Noise")

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()

    model.add(
        layers.Conv2D(
            156, (5, 5), strides=(2, 2), padding="same", input_shape=[78, 78, 3]
        )
    )
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(312, (5, 5), strides=(13, 13), padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()

In [ ]:
discriminator.summary()

In [ ]:
tf.keras.utils.plot_model(discriminator, show_shapes=True, to_file="out/discriminator.png")